# <h1 align="center"><font color="red">Text Classification with Text Embeddings</font></h1>

<font color="pink">Senior Data Scientist.: Eddy Giusepe Chirinos Isidro</font>

In [1]:
import openai

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay